Created on Thu Jan 24 16:50 2019

find the mean atmospheric effect on brightness temperature over sea ice in summer, to remove it from constant inferred in Figure 2

mask all SIC above 0.95 to 1.0 : 20190125-1448

@author: Clara Burgard

In [ ]:
import xarray as xr
import numpy as np

In [ ]:
import sys
sys.path.append('./arc3o')
import core_functions as arc3o
import arc3o_analysis_functions as sana

READ IN THE DATA

In [ ]:
inputpath = '/work/mh0033/m300411/SatSim/WORK_DATA/assimilation_runs/assim_SICCI2_50km/'
inputpath0 = '/work/mh0033/m300411/SatSim/WORK_DATA/assimilation_runs/assim_SICCI2_50km_mergetime/'
outputpath0 = '/work/mh0033/m300411/SatSim/WORK_DATA/simulated_TBs/'

In [ ]:
outputpath = arc3o.new_outputpath('no',outputpath0,'20190125-1448')

In [ ]:
orig_data = xr.open_dataset(inputpath0+'assim_SICCI2_50km_echam6_200211-200812_selcode_Arctic.nc')
#write time in the right format
orig_data = arc3o.prep_time(orig_data)

PREPARE THE INPUT DATA FOR EXPERIMENT

Set all melt pond fractions to zero to avoid their influence on the brightness temperature at the top of the atmosphere 

In [ ]:
orig_data['ameltfrac'] = orig_data['ameltfrac']*0

Set all sea-ice concentration above 99% to 100% to avoid the influence of open ocean on the brightness temperature at the top of the atmosphere but to keep the atmospheric state near to a full sea-ice cover

In [ ]:
orig_data['seaice'] = orig_data['seaice'].where(orig_data['seaice']<0.99,1.0)

RUN ARC3O ON THIS MODIFIED DATA

In [ ]:
start_year = 2003
end_year = 2008

file_begin = 'assim_SICCI2_50km_echam6_'
file_end = '_selcode_Arctic.nc'

freq_of_int = 6.9

arc3o.satsim_complete_parallel(orig_data,freq_of_int,start_year,end_year,inputpath,outputpath,file_begin,file_end,write_mask='no',write_profiles='yes',compute_memls='yes',e_bias=0.968)

CHECK RESULTS

The processing of the data is done in 'infer_atmos_influence_analysis.sh', resulting in the files 'TBVtot-TBV_assim_2003-2008_7_only99_ymonmean_fldmean.nc' and 'TBVtot-TBV_assim_2003-2008_7_only99_ymonmean_fldstd.nc'.

In [ ]:
#### with the cdo files
mean_atmo = xr.open_dataset(outputpath0+'20190125-1448/mergetime/TBVtot-TBV_assim_2003-2008_7_only99_ymonmean_fldmean.nc')
std_atmo = xr.open_dataset(outputpath0+'20190125-1448/mergetime/TBVtot-TBV_assim_2003-2008_7_only99_ymonstd_fldstd.nc')
mean_atmo_TBV = mean_atmo['TBV']
std_atmo_TBV = std_atmo['TBV']

for i in range(12):
    print('Atmos effect '+str(i+1)+' = '+str(np.round(mean_atmo_TBV.isel(time=i).values,2))+'+/-'+str(np.round(std_atmo_TBV.isel(time=i).values,2)))
    


Mean over the months at which bare wet ice could in theory be present

In [ ]:
mean_atmo_TBV.isel(time=[5,6,7,8]).mean()